In [99]:
import pandas as pd
import pickle
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.preprocessing import MinMaxScaler

# dataset is accessible at https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt (https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz)
df = pd.read_csv('amazon_reviews_us_Digital_Software_v1_00.tsv', sep='\t', dtype={'star_rating': float})

df.columns = ['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'star_rating',
              'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date']

df.drop(['marketplace', 'product_parent', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline'],
        axis='columns', inplace=True)

# Using just a subset of the data (the one with more common id in order for there to be more correlation)

review_counts = df['customer_id'].value_counts()
customers_with_multiple_reviews = review_counts[review_counts > 1].index
df = df[df['customer_id'].isin(customers_with_multiple_reviews)]

df = df.sort_values(by=['product_id', 'customer_id'])

# df = df[:10000]

print(df)

        customer_id       review_id  product_id  \
2793       14297784   R1RDB9C91XPF0  B000JLNHO6   
15096      19802240  R2MTDG51GV59MS  B000YMNI2Q   
99226      39503814  R2LRAB7GM83D56  B000YMNI2Q   
96894      14346072  R1YKWHT9ADKWWN  B000YMNI76   
101155     52744918  R20V03BFCC4Q76  B00194IWVK   
...             ...             ...         ...   
2643       34362164  R1XY7EM5DW79UE  B01019BOEA   
2268       44425668  R2CNQKNOLUH8ED  B01019BOEA   
3112       48216980  R3O4WQWL638OO0  B010909498   
4672       24267541  R172GVQEMEB3JA  B010BTET5Q   
1601        2917773  R313E51X4NISQS  B012Y7R126   

                                     product_title  product_category  \
2793                        TurboTax Business 2007  Digital_Software   
15096                          TurboTax Basic 2007  Digital_Software   
99226                          TurboTax Basic 2007  Digital_Software   
96894                 TurboTax Deluxe Federal 2007  Digital_Software   
101155  Better Homes and Ga

In [100]:
def apply_pivot(df, fillby=None):
    pivot_table = df.pivot_table(index='customer_id', columns='product_id', values='star_rating')
    if fillby is not None:
        pivot_table = pivot_table.fillna(fillby)
    return pivot_table

# Train-test split (training on older data to predict more recent data)
df = df.sort_values(by='review_date')

train_ratio = 0.7
split_index = int(train_ratio * len(df))

train = df[:split_index]
test = df[split_index:]

test = test[test.customer_id.isin(train.customer_id)] # to guarantee known customer ids in test

df_train_pivot = apply_pivot(df = train, fillby = 0)
df_test_pivot = apply_pivot(df = test, fillby = 0)

print(test.shape)

(1901, 8)


In [101]:
# Creating dummies to help to know wether a product has been rated or not
# Train
dummy_train = train.copy()

# Exclude products already rated by the user
# Obtain a table with 0 when products have been rated and 1 when they haven't
dummy_train['star_rating'] = dummy_train['star_rating'].apply(lambda x: 0 if x >= 1 else 1) 
dummy_train = apply_pivot(df = dummy_train, fillby = 1)

# Exclude products not rated by the user
# Obtain a table with 1 when products have been rated and 0 when they haven't
dummy_test = test.copy()
dummy_test['star_rating'] = dummy_test['star_rating'].apply(lambda x: 1 if x >= 1 else 0)
dummy_test = apply_pivot(df = dummy_test, fillby = 0)

In [102]:
# Calculate mean and subtract it from ratings for ajusted cosine similarity
train_pivot = apply_pivot(df=train)
mean = train_pivot.mean(axis=1, skipna=True)

df_train_subtracted = train_pivot.sub(mean, axis=0)

# Set ratings to 0 where a user hasn't given any rating
df_train_subtracted.fillna(0, inplace=True)

In [103]:
# Create the User Similarity Matrix
user_correlation = 1 - pairwise_distances(df_train_subtracted, metric='cosine')
user_correlation[np.isnan(user_correlation)] = 0

user_correlation_df = pd.DataFrame(user_correlation, index=df_train_subtracted.index, columns=df_train_subtracted.index)

#print(user_correlation_df.head())

In [104]:
# Rating predicted by the user is weighted sum of correlation with the product rating 
user_predicted_ratings = np.dot(user_correlation, df_train_pivot)

# Products not rated by user 
user_final_rating = np.multiply(user_predicted_ratings,dummy_train)

In [105]:
#Filter user correlation only for user which is in test, test is subset/equal of train in terms of userId

user_correlation_test_df = user_correlation_df[user_correlation_df.index.isin(test.customer_id)]
user_correlation_test_df = user_correlation_test_df[list(set(test.customer_id))]

#Get test user predicted rating
test_user_predicted_ratings = np.dot(user_correlation_test_df, df_test_pivot)
test_user_predicted_ratings = np.multiply(test_user_predicted_ratings,dummy_test)
test_user_predicted_ratings = test_user_predicted_ratings[test_user_predicted_ratings > 0]

scaler = MinMaxScaler(feature_range=(1, 5))
scaler.fit(test_user_predicted_ratings)
test_user_predicted_ratings = scaler.transform(test_user_predicted_ratings)

C:\Users\Fisica1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\preprocessing\_data.py:473: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\Fisica1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\preprocessing\_data.py:474: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [106]:
# Calculate RMSE

total_non_nan = np.count_nonzero(~np.isnan(test_user_predicted_ratings))
rmse = (np.sum(np.sum((df_test_pivot - test_user_predicted_ratings)**2))/total_non_nan)**0.5
print(rmse)

2.4548849040095906


In [107]:
# Calculate precision at k

def precision_at_k(actual, predicted, k):
    precision_values = []
    for user in actual.index:
        actual_items = actual.loc[user]  # Actual user preferences
        predicted_items = predicted.loc[user].nlargest(k).index  # Top K recommended items
        num_relevant = len(set(actual_items) & set(predicted_items))  # Number of relevant items
        precision = num_relevant / k  # Precision at K
        precision_values.append(precision)
    average_precision = np.mean(precision_values)
    return average_precision

k = 10

precision = precision_at_k(df_test_pivot, user_final_rating, 5)
print("Precision at", k, ":", precision)

Precision at 1000 : 0.0


In [108]:
pickle.dump(user_final_rating,open('./user_final_rating.pkl','wb'))